In [1]:
# # 도시명 집계함수 사ㅏ용 불가능 -> 오브젝트 제거 
# # 그룹하는데 기준할땐 사용할수있음 
# df_near = df_near.drop('도시명',axis=1).groupby(['군집','일시']).mean().reset_index()
# # 군집과 일시로 묶음 
# # 군집별일시
# 멀라시팔

# from functools import reduce
# import pandas as pd
# dfs = [df_coldwave, df_kr, df_near]
# cold_kr_near = reduce(lambda left, right: pd.merge(left,right,left_index=True, right_index=True), dfs)

# 일시.value_counts() 한다음에 일시는 무조건 1개만 있어야함..
# 일시가 여러개일경우 오브젝트 칼럼 의심
# 일시와 groupby사용해서 오브젝트컬럼 묶어줌...
# 그다음 여러개의 값의 개수가있는 오브젝트 컬럼을 옆으로 붙여줌. 
# 컬럼을 옆으로 붙인 데이터 저장. 
# 머신러닝을 위한 데이터들 따로저장
# 재난과 기후 
# 피쳐임포턴스를 시각화 하여 중요한 피쳐만 뽑아서 모델돌려보기 . 
# 한파- 주변국/한파- 한국/한파- 해상/한파- 북한 .. 근데 북한은 한국과 합치는게 낫지않을까 . 
# 한파 -한국,주변국 /한파 - 한국,북한/ 한파 -한국, 해상 

In [2]:
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
import pandas as pd 
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import os
from functools import reduce
import warnings 
warnings.filterwarnings('ignore')

In [3]:
# 분류모델 import 
# Logistic Regression,SGD,KNN,SVM,Decision Tree, Random Forest, LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report,roc_auc_score,f1_score

lr = LogisticRegression()
knn = KNeighborsClassifier()
svm = SVC()
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
lgbm = LGBMClassifier()

In [4]:
df_coldwave = pd.read_csv('./data/preprocessed/df_coldwave.csv',encoding='cp949',index_col=0)
df_kr = pd.read_csv('./data/preprocessed/국내기후(백령도제외).csv',index_col=0)
df_near = pd.read_csv('./data/preprocessed/preprocessed_주변국 기상데이터_199601to202108.csv',index_col=0)

In [8]:
df_coldwave
# 날짜 중복(지점때문에)
# 널값은 없음 
# 일시와 재난일어난날 object 

,연도,날짜,지점,일시,재난일어난날
0,1973,103,이천,1973-01-03,1973-01-03
1,1973,103,수원,1973-01-03,1973-01-03
2,1973,103,임실,1973-01-03,1973-01-03
3,1973,103,서산,1973-01-03,1973-01-03
4,1973,103,대관령,1973-01-03,1973-01-03
...,...,...,...,...,...
21982,2022,224,장수,2022-02-24,2022-02-24
21983,2022,224,대관령,2022-02-24,2022-02-24
21984,2022,224,인제,2022-02-24,2022-02-24
21985,2022,224,이천,2022-02-24,2022-02-24


In [12]:
df_kr
# 시간과 관측지점 object
# 시간이름 일시 
# 널값없음 

,시간,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,합계 일사,일 최심신적설,일 최심적설,합계 3시간 신적설,평균 전운량,평균 중하층운량,평균 지면온도,합계 대형증발량,합계 소형증발량,안개 계속 시간
0,1990-01-01,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,5.0,...,4.58,0.0,0.0,0.0,3.5,3.6,-1.5,2.721902,0.9,0.0
1,1990-01-02,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,4.8,...,3.44,0.1,0.1,0.1,3.5,3.8,-2.9,2.721902,0.8,0.0
2,1990-01-03,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,5.0,...,7.63,0.0,0.0,0.0,0.0,0.0,-4.4,2.721902,1.4,0.0
3,1990-01-04,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,3.0,...,5.84,0.0,0.0,0.0,0.5,1.8,-3.6,2.721902,1.0,0.0
4,1990-01-05,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,7.2,...,0.36,0.6,0.6,0.6,9.0,3.6,-0.6,2.721902,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11683,2021-12-27,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,6.8,...,1.37,0.5,1.2,0.5,8.1,8.1,3.9,1.000000,1.4,0.0
11684,2021-12-28,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,5.3,...,0.91,0.0,0.0,0.0,8.3,8.0,5.6,0.900000,1.3,0.0
11685,2021-12-29,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,7.9,...,4.94,0.0,0.0,0.0,8.5,7.8,6.9,2.300000,3.3,0.0
11686,2021-12-30,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,10.1,...,4.85,0.0,0.0,0.0,8.0,8.0,5.9,2.300000,3.2,0.0


In [22]:
df_near.isnull().sum()
# 인덱스가 9370인데 rows가 271875가 나왔으면 중복값이 있다는건데 이걸 어케아냐면 의심가는 칼럼의 value_counts()해보면 각 밸류가 몇개나있는지알수잇음
# 여튼 지역별 날짜 중복됨 예를들어 도쿄도 1월1일 창진도 1월1일 이런식으로 
# 군집 도시명 일시 object
# 널값없음 
# 저 중복값을 없애기위해 군집과 날짜를 groupby함. 군집별 날짜 이렇게 

도시명       0
일시        0
평균기온      0
최고기온      0
최저기온      0
습도        0
풍향        0
풍속        0
강수량       0
해면기압      0
이슬점 온도    0
군집        0
dtype: int64

In [23]:
# df_coldwave 전처리 

In [26]:
# 중복값제거할수없게만드는 column 삭제
df_coldwave.drop('지점',axis=1,inplace=True)

In [28]:
# 중복값제거함으로써 똑같은날짜 없앰
df_coldwave.drop_duplicates(inplace=True)

In [30]:
df_coldwave.drop(['연도','날짜'],axis=1,inplace=True)
#연도날짜칼럼 삭제

In [31]:
df_coldwave

,일시,재난일어난날
0,1973-01-03,1973-01-03
16,1973-01-04,1973-01-04
18,1973-01-06,1973-01-06
19,1973-01-11,1973-01-11
22,1973-01-12,1973-01-12
...,...,...
21950,2022-02-20,2022-02-20
21952,2022-02-21,2022-02-21
21960,2022-02-22,2022-02-22
21963,2022-02-23,2022-02-23


In [32]:
# 한국기후 전처리

In [33]:
df_kr
# 시간과 관측지점 object
# 시간이름 일시 
# 널값없음 

,시간,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,합계 일사,일 최심신적설,일 최심적설,합계 3시간 신적설,평균 전운량,평균 중하층운량,평균 지면온도,합계 대형증발량,합계 소형증발량,안개 계속 시간
0,1990-01-01,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,5.0,...,4.58,0.0,0.0,0.0,3.5,3.6,-1.5,2.721902,0.9,0.0
1,1990-01-02,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,4.8,...,3.44,0.1,0.1,0.1,3.5,3.8,-2.9,2.721902,0.8,0.0
2,1990-01-03,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,5.0,...,7.63,0.0,0.0,0.0,0.0,0.0,-4.4,2.721902,1.4,0.0
3,1990-01-04,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,3.0,...,5.84,0.0,0.0,0.0,0.5,1.8,-3.6,2.721902,1.0,0.0
4,1990-01-05,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,7.2,...,0.36,0.6,0.6,0.6,9.0,3.6,-0.6,2.721902,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11683,2021-12-27,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,6.8,...,1.37,0.5,1.2,0.5,8.1,8.1,3.9,1.000000,1.4,0.0
11684,2021-12-28,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,5.3,...,0.91,0.0,0.0,0.0,8.3,8.0,5.6,0.900000,1.3,0.0
11685,2021-12-29,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,7.9,...,4.94,0.0,0.0,0.0,8.5,7.8,6.9,2.300000,3.3,0.0
11686,2021-12-30,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,10.1,...,4.85,0.0,0.0,0.0,8.0,8.0,5.9,2.300000,3.2,0.0


In [34]:
# 시간컬럼을 일시로 이름바꾸기 
df_kr.rename(columns={'시간':'일시'},inplace=True)

In [35]:
df_kr

,일시,관측지점,평균 기온,최저 기온,최고 기온,1시간 최다강수량,일강수량,최대 순간풍속,최대 순간 풍속 풍향,최대 풍속,...,합계 일사,일 최심신적설,일 최심적설,합계 3시간 신적설,평균 전운량,평균 중하층운량,평균 지면온도,합계 대형증발량,합계 소형증발량,안개 계속 시간
0,1990-01-01,서울,-1.7,-4.0,1.8,0.0,0.0,7.3,270.0,5.0,...,4.58,0.0,0.0,0.0,3.5,3.6,-1.5,2.721902,0.9,0.0
1,1990-01-02,서울,-3.5,-5.7,-0.9,0.0,0.1,8.6,290.0,4.8,...,3.44,0.1,0.1,0.1,3.5,3.8,-2.9,2.721902,0.8,0.0
2,1990-01-03,서울,-5.0,-7.2,-1.7,0.0,0.0,8.2,290.0,5.0,...,7.63,0.0,0.0,0.0,0.0,0.0,-4.4,2.721902,1.4,0.0
3,1990-01-04,서울,-3.3,-8.9,2.1,0.0,0.0,6.1,290.0,3.0,...,5.84,0.0,0.0,0.0,0.5,1.8,-3.6,2.721902,1.0,0.0
4,1990-01-05,서울,0.7,-2.4,2.1,0.0,3.2,12.3,290.0,7.2,...,0.36,0.6,0.6,0.6,9.0,3.6,-0.6,2.721902,1.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11683,2021-12-27,제주,4.1,0.8,6.5,0.0,0.1,12.8,20.0,6.8,...,1.37,0.5,1.2,0.5,8.1,8.1,3.9,1.000000,1.4,0.0
11684,2021-12-28,제주,7.2,5.9,8.4,0.0,0.0,9.1,360.0,5.3,...,0.91,0.0,0.0,0.0,8.3,8.0,5.6,0.900000,1.3,0.0
11685,2021-12-29,제주,9.3,6.6,11.5,0.0,0.4,12.8,340.0,7.9,...,4.94,0.0,0.0,0.0,8.5,7.8,6.9,2.300000,3.3,0.0
11686,2021-12-30,제주,6.9,5.8,8.4,0.0,0.0,18.1,340.0,10.1,...,4.85,0.0,0.0,0.0,8.0,8.0,5.9,2.300000,3.2,0.0
